In [17]:
import cv2
import os

INPUT_DIR = "fotos"
OUTPUT_DIR = "fotos_binarizadas"

os.makedirs(OUTPUT_DIR, exist_ok=True)

for subdir, _, files in os.walk(INPUT_DIR):
    rel_path = os.path.relpath(subdir, INPUT_DIR)
    out_subdir = os.path.join(OUTPUT_DIR, rel_path)
    os.makedirs(out_subdir, exist_ok=True)

    for file in files:
        if file.lower().endswith(".jpg"):
            in_path = os.path.join(subdir, file)
            out_path = os.path.join(out_subdir, file)

            img = cv2.imread(in_path)

            if img is None:
                print(f"⚠️ No se pudo leer {in_path}")
                continue

            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            _, binary = cv2.threshold(
                gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU
            )

            cv2.imwrite(out_path, binary)
            print(f"✅ Procesada: {in_path} -> {out_path}")

print("✨ Proceso completado")

✅ Procesada: fotos/b5e380ff-1637-4cf5-9f52-1f3bede2c3ee/28.jpg -> fotos_binarizadas/b5e380ff-1637-4cf5-9f52-1f3bede2c3ee/28.jpg
✅ Procesada: fotos/b5e380ff-1637-4cf5-9f52-1f3bede2c3ee/29.jpg -> fotos_binarizadas/b5e380ff-1637-4cf5-9f52-1f3bede2c3ee/29.jpg
✅ Procesada: fotos/b5e380ff-1637-4cf5-9f52-1f3bede2c3ee/35.jpg -> fotos_binarizadas/b5e380ff-1637-4cf5-9f52-1f3bede2c3ee/35.jpg
✅ Procesada: fotos/b5e380ff-1637-4cf5-9f52-1f3bede2c3ee/34.jpg -> fotos_binarizadas/b5e380ff-1637-4cf5-9f52-1f3bede2c3ee/34.jpg
✅ Procesada: fotos/b5e380ff-1637-4cf5-9f52-1f3bede2c3ee/36.jpg -> fotos_binarizadas/b5e380ff-1637-4cf5-9f52-1f3bede2c3ee/36.jpg
✅ Procesada: fotos/b5e380ff-1637-4cf5-9f52-1f3bede2c3ee/33.jpg -> fotos_binarizadas/b5e380ff-1637-4cf5-9f52-1f3bede2c3ee/33.jpg
✅ Procesada: fotos/b5e380ff-1637-4cf5-9f52-1f3bede2c3ee/27.jpg -> fotos_binarizadas/b5e380ff-1637-4cf5-9f52-1f3bede2c3ee/27.jpg
✅ Procesada: fotos/b5e380ff-1637-4cf5-9f52-1f3bede2c3ee/26.jpg -> fotos_binarizadas/b5e380ff-1637-4cf5-9

In [19]:
import cv2, os, numpy as np
from math import pi

INPUT_DIR = "fotos"
OUTPUT_DIR = "fotos_binarizadas_limpias"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# parámetros
LEFT_MASK_FRAC = 0.08        # enmascara el 8% izquierdo (cámbialo si hay borde brillante)
BLUR_KSIZE = 7               # suavizado para reducir ruido (impar)
OPEN_K = 5                   # apertura morfológica para quitar puntos sueltos
AREA_MIN = 150               # descarta blobs muy pequeños
CIRC_MIN = 0.5               # circularidad mínima 4πA/P^2 (0..1)

def keep_largest_circular(blob_mask):
    # devuelve una máscara con el blob más grande y más circular
    cnts, _ = cv2.findContours(blob_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    best = None
    best_score = 0.0
    h, w = blob_mask.shape
    for c in cnts:
        area = cv2.contourArea(c)
        if area < AREA_MIN: 
            continue
        peri = cv2.arcLength(c, True) or 1.0
        circ = (4.0 * pi * area) / (peri * peri)  # 1.0 = círculo perfecto
        score = area * (0.5 + 0.5 * min(max(circ, 0.0), 1.0))  # pondera por circularidad y área
        if circ >= CIRC_MIN and score > best_score:
            best = c
            best_score = score
    out = np.zeros((h, w), np.uint8)
    if best is not None:
        cv2.drawContours(out, [best], -1, 255, thickness=-1)
    return out

for subdir, _, files in os.walk(INPUT_DIR):
    rel = os.path.relpath(subdir, INPUT_DIR)
    out_sub = os.path.join(OUTPUT_DIR, rel)
    os.makedirs(out_sub, exist_ok=True)

    for f in files:
        if not f.lower().endswith(".jpg"):
            continue
        in_path  = os.path.join(subdir, f)
        out_path = os.path.join(out_sub, f)

        img = cv2.imread(in_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print("⚠️ No se pudo leer", in_path); continue

        h, w = img.shape

        # (1) Enmascara franja izquierda si hay luz parásita
        if LEFT_MASK_FRAC > 0:
            m = int(w * LEFT_MASK_FRAC)
            bg = int(img[:, m:m+20].mean()) if m+20 < w else int(img.mean())
            img[:, :m] = bg

        # (2) Suavizado para reducir ruido
        blur = cv2.GaussianBlur(img, (BLUR_KSIZE, BLUR_KSIZE), 0)

        # (3) Otsu (la pelota es brillante → queremos binario con pelota blanca)
        _, bin0 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # (4) Morfología: apertura para quitar puntos sueltos
        k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (OPEN_K, OPEN_K))
        opened = cv2.morphologyEx(bin0, cv2.MORPH_OPEN, k, iterations=1)

        # (5) Quédate con el blob más grande y circular (la pelota)
        clean = keep_largest_circular(opened)

        cv2.imwrite(out_path, clean)
        print(f"✅ {in_path} -> {out_path}")

print("✨ Proceso completado")

✅ fotos/efdb9a78-d21c-4d28-9ffa-d18b0da9c036/17.jpg -> fotos_binarizadas_limpias/efdb9a78-d21c-4d28-9ffa-d18b0da9c036/17.jpg
✅ fotos/efdb9a78-d21c-4d28-9ffa-d18b0da9c036/21.jpg -> fotos_binarizadas_limpias/efdb9a78-d21c-4d28-9ffa-d18b0da9c036/21.jpg
✅ fotos/efdb9a78-d21c-4d28-9ffa-d18b0da9c036/20.jpg -> fotos_binarizadas_limpias/efdb9a78-d21c-4d28-9ffa-d18b0da9c036/20.jpg
✅ fotos/efdb9a78-d21c-4d28-9ffa-d18b0da9c036/22.jpg -> fotos_binarizadas_limpias/efdb9a78-d21c-4d28-9ffa-d18b0da9c036/22.jpg
✅ fotos/efdb9a78-d21c-4d28-9ffa-d18b0da9c036/23.jpg -> fotos_binarizadas_limpias/efdb9a78-d21c-4d28-9ffa-d18b0da9c036/23.jpg
✅ fotos/efdb9a78-d21c-4d28-9ffa-d18b0da9c036/27.jpg -> fotos_binarizadas_limpias/efdb9a78-d21c-4d28-9ffa-d18b0da9c036/27.jpg
✅ fotos/efdb9a78-d21c-4d28-9ffa-d18b0da9c036/26.jpg -> fotos_binarizadas_limpias/efdb9a78-d21c-4d28-9ffa-d18b0da9c036/26.jpg
✅ fotos/efdb9a78-d21c-4d28-9ffa-d18b0da9c036/18.jpg -> fotos_binarizadas_limpias/efdb9a78-d21c-4d28-9ffa-d18b0da9c036/18.jpg


In [18]:
import os

INPUT_DIR = "fotos"

# Lista de subcarpetas directas
subfolders = [d for d in os.listdir(INPUT_DIR) if os.path.isdir(os.path.join(INPUT_DIR, d))]

print(f"📂 Número de subcarpetas: {len(subfolders)}")
print("➡️ Subcarpetas encontradas:", subfolders)

📂 Número de subcarpetas: 84
➡️ Subcarpetas encontradas: ['efdb9a78-d21c-4d28-9ffa-d18b0da9c036', '716251da-fee2-4967-b9fe-7866a31ef411', 'f90398bd-2c43-48d0-9793-a6afa39af3f5', 'bb497379-c3fe-4c57-828e-50373b83e0ac', '6db94e04-74cb-40bb-8540-e6d85c131d4b', 'b3c46160-4170-4068-9216-840f5096412b', '3241265e-3cbf-451d-a755-8c52e1dd4f8d', '45de901b-4391-4ae6-9fe4-672102512339', '9ab94c7a-7a6c-4e6c-ac77-8cde10fdc113', 'b4b1ef4c-d73e-491e-9755-6ca84e27d09c', '7acc031f-90e4-4001-8a7d-4cfc6c206c56', '1dba53e2-be33-49b2-8786-5c9fe6991f24', 'ddf8b9be-e2ce-494e-80b2-308e389e52a7', 'f189c55a-d6dc-41e1-9cad-e08c35e03e0d', 'eba055ca-0b8c-4a67-ad40-5483d0cf60ac', '8f4239dc-3e76-4f54-9d90-529194fb529e', 'f25278b6-2880-484a-abf9-7bd4581761f1', '7caa7a78-a09e-4a82-b452-363fc25cf31c', '0844f10e-40c7-4661-9fdf-dd11d2c900a9', '88766972-f040-4462-9e19-4de63342f3bb', 'cfe58800-4ff8-426d-9db4-796068fc9d8a', 'ca6fb2d5-d31e-4285-b4b4-72d2449779fe', 'c811eb49-d8be-45f5-ae60-d049bca2463e', '6b9cafdb-7584-47d2-bb3